In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import ast

In [116]:
pd.options.display.max_columns = False


In [117]:
chunk = pd.read_csv('../../data/raw.csv', engine='c',sep=',', encoding='utf-8', on_bad_lines='skip', chunksize=11000)

In [118]:
df = chunk.get_chunk(11000)

In [119]:
selected_cols = ['title', 'createdAt',
       'cities', 'price', 'smallDescription', 'store.name',
]

In [120]:
df = df[selected_cols]

In [121]:
df['createdAt'] = pd.to_datetime(df['createdAt'])
df.price = pd.to_numeric(df.price, downcast='integer')

In [122]:
df.dtypes

title                            object
createdAt           datetime64[ns, UTC]
cities                           object
price                           float64
smallDescription                 object
store.name                       object
dtype: object

In [123]:
city = pd.json_normalize(df.cities.apply(ast.literal_eval).explode()).drop(columns=['id', 'slug', '__typename', 'region.id', 'region.slug', 'region.__typename' ]).rename(columns={'name':'Town', 'region.name':'Wilaya'})
area = pd.json_normalize(df.smallDescription.apply(ast.literal_eval).explode()).drop(columns= ['__typename']).rename(columns={'valueText': 'Area'})

In [124]:
area.Area = area.Area.astype(str)

In [125]:
def to_integer(s: str) -> pd.Int64Dtype(): 
    match = re.search(r'(\d+)', s or "")
    return int(match.group(1)) if match else pd.NA

area.Area = area.Area.apply(to_integer)

In [126]:
df = pd.concat([df, city, area], axis=1)

In [127]:
df = df.drop(columns=['cities', 'smallDescription'])

In [128]:
df = df.rename(columns ={'title': 'Title', 'createdAt': 'Date', 'store.name': 'Store', 'price': 'Price'})

In [129]:
df = df.iloc[:11000]

In [130]:
df['FlatType'] = df['Title'].str.extract(r'\b(F\d+)\b', expand=False)

In [131]:
mask = df['Title'].str.contains(r'\b(Location|Vente)\b', regex=True, na=False, case=False)

/tmp/ipykernel_8203/2649798588.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['Title'].str.contains(r'\b(Location|Vente)\b', regex=True, na=False, case=False)


In [132]:
df = df[~(mask == False)]

In [133]:
mask = df['Title'].str.contains(r'\b(Location)\b', regex=True, na=False, case=False)

/tmp/ipykernel_8203/1442584853.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['Title'].str.contains(r'\b(Location)\b', regex=True, na=False, case=False)


In [134]:
df_location = df[mask]
df_vente = df[~mask]

In [135]:
df_location.to_csv('../../data/processed_renting.csv', index=False)
df_vente.to_csv('../../data/processed_selling.csv', index=False)